In [1]:
import keras
from keras.layers.convolutional import MaxPooling3D, Conv3D
from keras.layers import Input, Dropout, Dense, Flatten
from keras.models import Model
from keras import optimizers
import logging

logging.basicConfig(level=logging.DEBUG)

Using TensorFlow backend.


In [2]:
class DummyGen(object):
    
    def __init__(self, width=3600, height=3600, labels=10):
        self.width = 3600
        self.height = 3600
        self.labels = 10
        
    @property
    def input(self):
        return self.labels
    
    @property
    def output(self):
        return (1, 3, self.height, self.width)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next
    
    def next(self):
        raise StopIteration("This is a dummy gen")
        
mygen = DummyGen()

In [3]:
class Minnet(Model):
  logger = logging.getLogger('pdk.minnet')

  def __init__(self, generator):

    self.generator = generator

    layer = self.assemble(generator)
    super(Kevnet, self).__init__(self._input, layer)
    self.logger.info("Compiling Model")
    self.sgd = optimizers.SGD(lr=0.01, 
                              decay=1e-6,
                              momentum=0.9, 
                              nesterov=True)
    self.compile(loss='mean_squared_error', optimizer=self.sgd,
                 metrics=['accuracy'])

  def assemble(self, generator):

    self.logger.info("Assembling Model")
    self._input = Input(shape=generator.output,
                        dtype='float32')
    self.logger.info(self._input)

    layer = Conv3D(32, (1, 3, 3), strides=(1, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first')(self._input)
    self.logger.info(layer)
    layer = MaxPooling3D((1, 3, 3), strides=(1, 2, 2),
                         data_format='channels_first')(layer)
    self.logger.info(layer)

    layer = Conv3D(64, (1, 3, 3), strides=(1, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first')(layer)
    self.logger.info(layer)
    layer = MaxPooling3D((1, 3, 3), strides=(1, 2, 2),
                         data_format='channels_first')(layer)
    self.logger.info(layer)

    layer = Conv3D(128, (1, 3, 3), strides=(1, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first')(layer)
    self.logger.info(layer)
    layer = MaxPooling3D((1, 3, 3), strides=(1, 2, 2),
                         data_format='channels_first')(layer)
    self.logger.info(layer)

    layer = Conv3D(512, (3, 3, 3), strides=(3, 2, 2),
                   activation='relu', padding='same',
                   data_format='channels_first')(layer)
    self.logger.info(layer)
    layer = MaxPooling3D((1, 3, 3), strides=(1, 2, 2),
                         data_format='channels_first')(layer)
    self.logger.info(layer)

    layer = Conv3D(1024, (1, 3, 3), strides=(1, 3, 3),
                   activation='relu', padding='same',
                   data_format='channels_first')(layer)
    self.logger.info(layer)
    layer = MaxPooling3D((1, 3, 3), strides=(1, 3, 3),
                         data_format='channels_first')(layer)
    self.logger.info(layer)
    self.last_conv_layer = layer

    # Classification block
    layer = Flatten(name='flatten')(layer)
    layer = Dropout(0.01)(layer)
    layer = Dense(2048, activation='relu')(layer)
    layer = Dropout(0.01)(layer)
    layer = Dense(generator.input,
                  activation='softmax')(layer)
    self.logger.info(layer)
    return layer


In [4]:
mymodel= Kevnet(mygen)

INFO:pdk.kevnet:Assembling Model
INFO:pdk.kevnet:Tensor("input_1:0", shape=(?, 1, 3, 3600, 3600), dtype=float32)
INFO:pdk.kevnet:Tensor("conv3d_1/Relu:0", shape=(?, 32, 3, 1800, 1800), dtype=float32)
INFO:pdk.kevnet:Tensor("max_pooling3d_1/transpose_1:0", shape=(?, 32, 3, 899, 899), dtype=float32)
INFO:pdk.kevnet:Tensor("conv3d_2/Relu:0", shape=(?, 64, 3, 450, 450), dtype=float32)
INFO:pdk.kevnet:Tensor("max_pooling3d_2/transpose_1:0", shape=(?, 64, 3, 224, 224), dtype=float32)
INFO:pdk.kevnet:Tensor("conv3d_3/Relu:0", shape=(?, 128, 3, 112, 112), dtype=float32)
INFO:pdk.kevnet:Tensor("max_pooling3d_3/transpose_1:0", shape=(?, 128, 3, 55, 55), dtype=float32)
INFO:pdk.kevnet:Tensor("conv3d_4/Relu:0", shape=(?, 512, 1, 28, 28), dtype=float32)
INFO:pdk.kevnet:Tensor("max_pooling3d_4/transpose_1:0", shape=(?, 512, 1, 13, 13), dtype=float32)
INFO:pdk.kevnet:Tensor("conv3d_5/Relu:0", shape=(?, 1024, 1, 5, 5), dtype=float32)
INFO:pdk.kevnet:Tensor("max_pooling3d_5/transpose_1:0", shape=(?, 10